In [1]:
# pip install gspread streamlit pandas dotenv

In [2]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
import streamlit as st

scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

# Load from Streamlit secrets
service_account_info = st.secrets["gcp_service_account"]
creds = Credentials.from_service_account_info(service_account_info, scopes=scope)
client = gspread.authorize(creds)

# Open your sheet
sheet = client.open("f87_rag_logs").sheet1
data = sheet.get_all_records()
df = pd.DataFrame(data)


2025-05-17 22:00:41.466 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            


In [3]:
df.head()

,Timestamp,question,answer,Context1,Context2,Context3,Context4,Context5,Context6
0,2025-05-15 20:42:56,What is the VANOS on the M2?,The VANOS system on the BMW M2 is a variable v...,Interesting comparison of the engine managemen...,a bit exposes. The front engine harness needs...,"1. Transcript of the ""Driving stability contro...",off eBay. He installed it. And was getting a e...,Hey all - I recently received and installed th...,"Anyone know if Valve Control devices (VSC-2, B..."
1,2025-05-15 20:44:41,Is it possible to adjust the VANOS?,"No, the VANOS system itself is not adjustable ...",to control them manually\n\nI notice a pretty ...,I'm facing a weird issue with the VSC-2 where ...,types of ticking/clicking noises.1. VANOS rela...,Since it seems this is now adjustable if using...,would be better for throttle response as its n...,", there is CG Precision VSC-1 that is plug-n-p..."
2,2025-05-15 20:52:13,What oil should I use?,"For your M2 LCI with the N55 engine, consideri...",repackaged Shell. Shell owns Pennzoil and they...,So from my research thus far this is a short l...,The car has many many miles beyond the break i...,I'm looking to do an oil change for the start ...,"Hi all,My M2 LCI (N55) now has around 8000 mil...",hetics and have extremely low pour point and v...
3,2025-05-16 04:19:15,What suspension components are recommend to up...,"For heavy track use, consider upgrading the fo...",You might chat with TCK to see what they recom...,"After 2 years of tracking with stock 18 M2, am...","Hi, I'm specifically posting this here because...",The OEM suspension seems great for the track b...,I am struggling with deciding what my next pur...,"Hi guys, what suspension components would you ..."
4,2025-05-16 04:20:15,What are some recommended camber plates?,"For your BMW M2, here are some recommended cam...",Does anyone know what the OEM camber specs are...,What's your recommended Camber settings for DD...,negative camber on the front so the factory st...,Would what you suggested be the max before pok...,I can't stand tracking this car anymore with t...,Hi folks. Im looking at getting a set of Ohli...


In [4]:
context_list = []

for index in df.iterrows():
    cont = df.iloc[index[0],3:6].to_list()
    context_list.append(cont)


In [5]:
df["contexts"] = context_list

In [6]:
import os
from dotenv import load_dotenv
from ragas import evaluate
from datasets import Dataset, Features, Value, Sequence

# Define schema
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Sequence(Value("string"))
})

# Convert to Dataset with schema
ragas_dataset = Dataset.from_pandas(df[["question", "answer", "contexts"]], features=features)


c:\Users\mattr\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain.chat_models import ChatOpenAI
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# Set your OpenAI API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Run evaluation using ragas.evaluate()
results = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        # context_precision,
        # context_recall,
    ],
    llm=llm,
)

print(results)


C:\Users\mattr\AppData\Local\Temp\ipykernel_19232\4150431211.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
Evaluating: 100%|██████████| 122/122 [00:54<00:00,  2.22it/s]


{'faithfulness': 0.3095, 'answer_relevancy': 0.8160}
